In [ ]:
import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
import seaborn as sns
import json
import h5py

import ipfx.script_utils as su
from ipfx.stimulus import StimulusOntology
import allensdk.core.json_utilities as ju
import ipfx.time_series_utils as tsu
import ipfx.data_set_features as dsf
from ipfx.dataset.create import create_ephys_data_set

import scipy.stats as stats
import statsmodels.stats.multitest as multitest

from scipy.ndimage import gaussian_filter1d

In [ ]:
tx_anno_file = ("/data/mouse_patchseq_VISp_20210818_collapsed40_cpm/anno.feather")
tx_data_file = ("/data/mouse_patchseq_VISp_20210818_collapsed40_cpm/data.feather")

spca_df = pd.read_csv("/data/spca_output/sparse_pca_components_mMET_exc_visp_Nov2021_flex.csv", index_col=0)
spec_ids = spca_df.index.values

In [ ]:
spca_df.shape

In [ ]:
ps_anno = pd.read_feather(tx_anno_file)
ps_anno["spec_id_label"] = pd.to_numeric(ps_anno["spec_id_label"])
ps_anno.set_index("spec_id_label", inplace=True)
ps_anno = ps_anno.loc[spec_ids, :]

In [ ]:
met_type_df = pd.read_csv("/data/exc_mouse_met/derived_data/exc_met_cell_text_assignments.csv", index_col=0)
met_type_df.shape

In [ ]:
morph_df = pd.read_csv("/data/exc_mouse_met/derived_data/morph_features_mMET_exc_wide_raw.csv", index_col=0)

In [ ]:
l5et_met_types = ("PT-MET-1", "PT-MET-2", "PT-MET-3")
l5et_met_cell_ids = met_type_df.index[met_type_df["met_type"].isin(l5et_met_types)]
l5et_met_cell_ids

In [ ]:
np.savetxt("/results/l5et_met_cell_ids.txt", l5et_met_cell_ids, fmt="%d")

In [ ]:
l5et_morph_df = morph_df.loc[l5et_met_cell_ids, :].copy()
l5et_morph_df['met_type'] = met_type_df.loc[l5et_met_cell_ids, "met_type"].values

In [ ]:
np.sort(ps_anno.Tree_first_cl_label.unique())

In [ ]:
l5et_all_cells = ps_anno.loc[
    ps_anno.Tree_first_cl_label.isin(['L5 PT VISp C1ql2 Cdh13',
       'L5 PT VISp C1ql2 Ptgfr', 'L5 PT VISp Chrna6', 'L5 PT VISp Krt80',
       'L5 PT VISp Lgr5']) & ps_anno.Tree_call_label.isin(["Core", "I1", "I2", "I3"]), :].index.values

In [ ]:
np.savetxt("/results/l5et_all_cell_ids.txt", l5et_all_cells, fmt="%d")

In [ ]:
len(l5et_all_cells)

In [ ]:
met_type_colors = {
    'IT-MET-1': '#7AE6AB',
    'IT-MET-2': '#00979D',
    'IT-MET-3': '#00DDC5',
    'IT-MET-4': '#00A809',
    'IT-MET-5': '#26BF64',
    'IT-MET-6': '#96E32C',
    'IT-MET-7': '#A19922',
    'IT-MET-8': '#5100FF',
    'PT-MET-1': '#0000FF',
    'PT-MET-2': '#22737F',
    'PT-MET-3': '#29E043',
    'L6b-MET-1': '#2B7880',
    'L6b-MET-2': '#254566',
    'L6b-MET-3': '#336D99',
    'NP-MET-1': '#73CA95',
    'CT-MET-1': '#74CAFF',
}

In [ ]:
ax = sns.scatterplot(
    data=l5et_morph_df,
    x="apical_dendrite_extent_y",
    y="apical_dendrite_num_branches",
    hue="met_type",
    palette=met_type_colors)
sns.despine()
ax.set_ylabel("apical dendrite\nnumber of branches")
ax.set_xlabel("apical dendrite - height (µm)")
plt.savefig("/results/l5et_met_morph_characteristics.png", bbox_inches='tight', dpi=300)

In [ ]:
rate_df = pd.read_csv("/data/exc_mouse_met/scripts/l5et_avg_inst_rates.csv", index_col=0)
rate_df

In [ ]:
rate_df["ratio"] = rate_df['max_inst_rate'] / rate_df['avg_rate']

In [ ]:
max_ratios = rate_df.reset_index().groupby("specimen_id")["ratio"].max()

In [ ]:
met_type_df

In [ ]:
rate_df['met_type'] = ""
met_cells = met_type_df.index.intersection(rate_df.index)
rate_df.loc[met_cells, 'met_type'] = met_type_df.loc[met_cells, 'met_type']

In [ ]:
rate_df

In [ ]:
rate_df["t_type"] = ps_anno.loc[rate_df.index, "Tree_first_cl_label"]

In [ ]:
rate_df

In [ ]:
autotrace_met_type_df = pd.read_csv("/data/exc_mouse_met/derived_data/autotrace_and_manual_exc_met_cell_assignments.csv", index_col=0)

In [ ]:
assign_met_by_ttype = {
    'L2/3 IT VISp Agmat': "IT-MET-1",
    'L2/3 IT VISp Rrad': "IT-MET-1",
    'L2/3 IT VISp Adamts2': "IT-MET-1",
    'L4 IT VISp Rspo1': "IT-MET-2",
    'L5 IT VISp Hsd11b1 Endou': "IT-MET-3",
    'L5 IT VISp Whrn Tox2': "IT-MET-3",
    'L5 IT VISp Col6a1 Fezf2': "IT-MET-4",
    'L5 IT VISp Batf3': "IT-MET-4",
    'L5 IT VISp Col27a1': "IT-MET-5",
    'L6 IT VISp Col18a1': "IT-MET-6",
    'L6 IT VISp Car3': "IT-MET-8",
    'L5 PT VISp Chrna6': "PT-MET-1",
    'L5 PT VISp C1ql2 Cdh13': "PT-MET-3",
    'L5 PT VISp C1ql2 Ptgfr': "PT-MET-3",
    'L5 PT VISp Krt80': "PT-MET-3",
    'L5 NP VISp Trhr Met': "NP-MET-1",
    'L5 NP VISp Trhr Cpne7': "NP-MET-1",
    'L6 CT VISp Ctxn3 Brinp3': "CT-MET-1",
    'L6 CT VISp Ctxn3 Sla': "CT-MET-1",
    'L6 CT VISp Nxph2 Wls': "CT-MET-1",
    'L6 CT VISp Krt80 Sla': "CT-MET-1",
    'L6 CT VISp Gpr139': "CT-MET-1",
    'L6b VISp Mup5': "L6b-MET-1",
    'L6b Col8a1 Rprm': "L6b-MET-1",
    'L6b VISp Col8a1 Rxfp1': "L6b-MET-1", 
    'L6b VISp Crh': "L6b-MET-2", 
    'L6b P2ry12': "L6b-MET-3",
}

In [ ]:
autotrace_to_met = {
    0: "PT-MET-3",
    3: "PT-MET-3",
    5: "PT-MET-2",
    10: "IT-MET-7",
    4: "IT-MET-6",
    11: "IT-MET-6",
}

In [ ]:
rate_mask = (rate_df['ratio'].values == max_ratios[rate_df.index])

In [ ]:
rate_maxratio_df = rate_df.loc[rate_mask.values, :].copy()

In [ ]:
rate_maxratio_df.shape

In [ ]:
inferred_met_types = []
for spid in rate_maxratio_df.index.tolist():
    if rate_maxratio_df.at[spid, "met_type"] != "":
        inferred_met_types.append(rate_maxratio_df.at[spid, "met_type"])
    elif rate_maxratio_df.at[spid, "t_type"] in assign_met_by_ttype:
        inferred_met_types.append(assign_met_by_ttype[rate_maxratio_df.at[spid, "t_type"]])
        print("got", assign_met_by_ttype[rate_maxratio_df.at[spid, "t_type"]])
    elif spid in autotrace_met_type_df.index.tolist():
        autotrace_met_type = autotrace_met_type_df.at[spid, "met_type"]
        if autotrace_met_type in autotrace_to_met:
            print("inferring by autotrace")
            inferred_met_types.append(autotrace_to_met[autotrace_met_type])
        else:
            inferred_met_types.append("")
    else:
        inferred_met_types.append("")

In [ ]:
rate_maxratio_df['inferred_met_type'] = inferred_met_types

In [ ]:
rate_maxratio_df

In [ ]:
rate_maxratio_df["marker_size"] = 1
rate_maxratio_df.loc[rate_maxratio_df.met_type != "", 'marker_size'] = 2

In [ ]:
rate_maxratio_df

In [ ]:
met_type_colors[""] = "gray"

In [ ]:
ax = sns.scatterplot(
    data=rate_maxratio_df.loc[rate_maxratio_df.inferred_met_type.isin(["PT-MET-1", "PT-MET-2", "PT-MET-3"]), :],
    x='avg_rate',
    y='max_inst_rate',
    hue='inferred_met_type',
    size='marker_size',
    sizes=(10, 30),
    palette=met_type_colors,
    legend=False)
ax.set_ylabel("Maximum instantaneous rate (spikes/s)")
ax.set_xlabel("Average rate (spikes/s)")
ax.set_xlim(0, 25)
sns.despine()
plt.savefig("/results/l5et_met_bursting_characteristics.png", bbox_inches='tight', dpi=300)

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-1"), :]

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-1"), :].median()

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-2"), :]

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-2"), :].median()

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-3") & (rate_df['ratio'] > 20), :]

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-3") & (rate_df['ratio'] > 20), :].median()

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-3") & (rate_df['ratio'] < 20), :]

In [ ]:
rate_df.loc[rate_mask.values & (rate_df['met_type'] == "PT-MET-3") & (rate_df['ratio'] < 20), :].median()

In [ ]:
rate_df.groupby("met_type").get_group("PT-MET-1")

In [ ]:
rate_df.loc[971546167, :]

In [ ]:
ontology = StimulusOntology(ju.read(StimulusOntology.DEFAULT_STIMULUS_ONTOLOGY_FILE))